In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import csv
import os
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import sys
Stopwords = set(stopwords.words('english'))
porter = PorterStemmer()
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

!pip install textdistance
!pip install elasticsearch
from elasticsearch import Elasticsearch
import textdistance

lemmatizer = WordNetLemmatizer()
import time
import ast
from collections import Counter
import copy 
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 327kB 8.8MB/s 


Creating Server for Elastic Search

In [2]:
#Downloading Elastic Search

!apt install default-jdk > /dev/null
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-6.5.4.tar.gz -q --show-progress
!tar -xzf elasticsearch-6.5.4.tar.gz
!chown -R daemon:daemon elasticsearch-6.5.4
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-6.5.4/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# client-side
!pip install elasticsearch -q
from elasticsearch import Elasticsearch
es = Elasticsearch()



E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/o/openjdk-lts/openjdk-11-jdk_11.0.9+11-0ubuntu1~18.04.1_amd64.deb  404  Not Found [IP: 91.189.88.152 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
elasticsearch-6.5.4 100%[===================>] 108.07M  32.2MB/s    in 3.4s    


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [5]:
#Preprocessing

def get_terms(line):
    '''given a stream of text, get the terms from the text'''
    line = line.lower()
    line = re.sub(r'[^a-z0-9 ]', ' ', line)  # put spaces instead of non-alphanumeric characters
    line = re.sub(' +', ' ', line)
    words_tokens = word_tokenize(line)
    # words_tokens = [word for word in words_tokens if len(words_tokens)>1]
    line = [w for w in words_tokens if not w in Stopwords]
    return line

def preprocess(line):
    line = re.sub(r"(\.|,|\?|\(|\)|\[|\]|\!|\'|\||\%|\:)", " ", line)
    line = re.sub(r'[^a-z0-9 ]', ' ', line)  # put spaces instead of non-alphanumeric characters
    line = re.sub('(?<=[a-z])\'(?=[a-z])', '', line)
    line = line.replace("-", "")
    line = line.replace("gonna", "going to")
    return line

contractions = {
"ain't": "are not",
"aren't": "am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i had",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"I'am":"I am"
}

In [6]:
#path='/content/drive/My Drive/TelevisionNews'
path='/content/drive/My Drive/AIR/TelevisionNews'
all_files = glob.glob(os.path.join(path, "*.csv"))
#print(len(all_files))

In [7]:
#B-TREE implementation

keywords=[]
class Node:
    def __init__(self, data):

        self.left = None
        self.right = None
        self.data = data
    def insert(self, data):
# Compare the new value with the parent node
        if self.data:
            if data < self.data:
                if self.left is None:
                    self.left = Node(data)
                else:
                    self.left.insert(data)
            elif data > self.data:
                if self.right is None:
                    self.right = Node(data)
                else:
                    self.right.insert(data)
        else:
            self.data = data

    def PrintTree(self):
        if self.left:
            self.left.PrintTree()
        keywords.append(self.data)
        if self.right:
            self.right.PrintTree()

    def findval(self, lkpval):
        if lkpval < self.data:
            if self.left is None:
                return "0"
            return self.left.findval(lkpval) 
        elif lkpval > self.data:
            if self.right is None:
                return "0"
            return self.right.findval(lkpval)
        else:
            return self.data

In [8]:
#creation of inverted index 
inverted_index={}
tf={} #term Frequency
doc_count=0
doc_to_id={}
root=None
for file_no, file in enumerate(all_files):
    try:
      data_file= pd.read_csv(file)
      df = pd.DataFrame(data_file)
      df['Snippet'].dropna(inplace=True)
      data=df["Snippet"]
      tf[file]={}
      doc_to_id[file_no]=file
      for snippet_no, line in enumerate(data):
          doc_count+=1
          line=preprocess(line)
          for contraction in contractions:
                if contraction in line:
                    line=line.replace(contraction,contractions[contraction])
          line=lemmatizer.lemmatize(line)
          line_ref= word_tokenize(line)
          tf[file][snippet_no]={}
          terms=get_terms(line)
          words = line.split(' ')
          for term in terms:
              if term in words:
                  if not tf[file][snippet_no].get(term):
                    tf[file][snippet_no][term]=1
                  else:
                    tf[file][snippet_no][term]+=1
                  term_pos_list = [index for index, element in enumerate(terms) if element == term]

                  if term not in inverted_index.keys():
                      inverted_index[term]={}
                      inverted_index[term][file_no]={}
                      inverted_index[term][file_no][snippet_no]=term_pos_list
                  else:
                      try:
                          inverted_index[term][file_no][snippet_no]=term_pos_list
                      except:
                          inverted_index[term][file_no]={}
                          inverted_index[term][file_no][snippet_no]=term_pos_list

                  if root==None:
                      root = Node(term)
                  else:
                      root.insert(term)
    except:
      continue


In [9]:
#store the inverted index
with open(r"/content/drive/My Drive/AIR/res.txt",'w') as f:
    print(inverted_index, file=f)

In [10]:
file = open(r"/content/drive/My Drive/AIR/res.txt", "r")

contents = file.read()
inverted_index = ast.literal_eval(contents)

In [11]:
#Document Frequency
df={}
for doc in tf.keys():
  for snippet in tf[doc].keys():
    for term in tf[doc][snippet]:
      if df.get(term):
        df[term]+=1
      elif not df.get(term):
        df[term]=1
tf1 = copy.deepcopy(tf)
for doc in tf1.keys():
  for snippet in tf1[doc].keys():
    for term in tf1[doc][snippet]:
      l=len(tf1[doc][snippet])
      tf1[doc][snippet][term]=tf[doc][snippet][term]/l

In [12]:
#SPELL CHECK!!!
df1=Counter(df)
V=df1.keys()
probs = {}   
Total = max(df1.values()) 
for k in df1.keys():
    probs[k] = df1[k]/Total

def spell_check(input_word):
    input_word = input_word.lower()
    if input_word in V:
        return('Your word seems to be correct')
    else:
        similarities = [1-(textdistance.Jaccard(qval=2).distance(v,input_word)) for v in df1.keys()]
        df = pd.DataFrame(list(probs.items()),columns = ['word', 'Prob']) 
        df['Similarity'] = similarities
        output = df.sort_values(['Similarity', 'Prob'], ascending=False).head(3)
        return output

In [13]:
#wild card 
def isMatch(s, p):
      sl = len(s)
      pl = len(p)
      dp = [[False for i in range(pl+1)] for j in range(sl+1)]
      s = " "+s
      p = " "+p
      dp[0][0]=True
      for i in range(1,pl+1):
         if p[i] == '*':
            dp[0][i] = dp[0][i-1]
      for i in range(1,sl+1):
         for j in range(1,pl+1):
            if s[i] == p[j] or p[j] == '?':
               dp[i][j] = dp[i-1][j-1]
            elif p[j]=='*':
               dp[i][j] = max(dp[i-1][j],dp[i][j-1])
      return dp[sl][pl]
def wild_query(query):
    result=[]
    keywords=[]
    if ('*' in query):
        matched=[]
        for word in df1.keys():
            b=isMatch(str(word),query)
            if (b==True):
                keywords.append(word)
            else:
                continue
    for keyword in (keywords):
        res_term= root.findval(keyword)
        result.append(res_term)
    return result

Free text and Phrase QUery

In [47]:
def intersect(p1,p2):
    ans=[]
    p1len=len(p1)
    p2len=len(p2)
    i=0
    j=0
    while i!=p1len and j!=p2len:
        if p1[i]==p2[j]:
            ans.append(p1[i])
            i+=1
            j+=1
        else:
            if p1[i]<p2[j]:
                i+=1
            else:
                j+=1
    return ans

def intersect_prox(p1,p2,k):
    l=[]
    p1len=len(p1)
    p2len=len(p2)

    i=j=0
    while i!=p1len:
        while j!=p2len:
            if abs(p1[i]-p2[j])<=k:
                l.append(p2[j])
            elif p2[j]>p1[i]:
                break
            j+=1
        l.sort()
        while l!=[] and abs(l[0]-p1[i])>k:
            l.remove(l[0])
        i+=1
  
    return l

def sort_docs_list(docs_list):
    docs_list.sort(key=len)
    return docs_list


def extract_docs(term):
    docs_list=[]
    for i in inverted_index[term].keys():
        docs_list.append(i)
    return docs_list

def extract_snippets(term,doc_no):
    snippets_list=[]
    for i in inverted_index[term][doc_no].keys():
        snippets_list.append(i)
    return snippets_list


def extract_positional_indices(term,doc_no,snip_no):
    positions_list=[]
    for i in inverted_index[term][doc_no][snip_no]:
        positions_list.append(i)
    return positions_list

def phrase_proximity_freetext_query(query,inverted_index):
    results=[]
    if '/' in query:
        query_new = re.sub(r'[0-9]+', '', query)
        query_new = preprocess(query_new)
    else:
        query_new = preprocess(query)
    
    terms = get_terms(query_new)
    words = terms
    exact_words=  word_tokenize(query_new)

    total_docs={}
    for keyword in range(len(words)):
        res_term= root.findval(words[keyword])
        total_docs[res_term]=extract_docs(res_term)

    
    total_docs_list= list(total_docs.values())
    total_docs_list= sort_docs_list(total_docs_list)
    l=total_docs_list[0]
    for i in total_docs_list:
        l=intersect(l,i)

    if l:
        for doc in l:
            total_snips={}
            for term in list(total_docs.keys()):
                total_snips[term]=extract_snippets(term,doc)

            total_snips_list = list(total_snips.values())
         
            total_snips_list= sort_docs_list(total_snips_list)
            s=total_snips_list[0]
            for i in total_snips_list:
                s=intersect(s,i)

            if s:
                for snip in s:
                    total_positions={}
                    for term in list(total_docs.keys()):
                        total_positions[term]=extract_positional_indices(term,doc,snip)

                    if '/' not in query and  "\"" in query:
                        # print("Query Type: Phrase Query")

                        if len(exact_words)==1:
                            # print('doc:snip:positions:',doc,snip,total_positions[exact_words[0]])
                            # print("Phrase Query: found in document {}, snippet {}".format(doc,snip))
                            results.append([doc,snip])

                        for i in range(len(exact_words)-1):
                            p1= total_positions[exact_words[i]]
                            p2= total_positions[exact_words[i+1]]
                      
                            for i in p1:  
                                if i+1 in p2:
                                    results.append([doc,snip])
                                    # print("Phrase Query: found in document {}, snippet {}".format(doc,snip))
                                else:
                                    pass
                                    # print('no results')

                    elif '/' in query :
                        # print("Query Type: Proximity Query")

                        proximity = re.findall(r'\d+' , query)
                        query1 = query.split(" ")
                        query_new1= query_new.split(" ")
                        token = []
                        terms=[]
                        for i in query_new1:
                            if '/' not in i and i!='':
                                terms.append(i)
                        token.append(query1[0])
                   
                        for i in range(0,len(query1)):
                         
                            if ("/" in query1[i]):
                                k = int(proximity[0])
                                p1=total_positions[token[0]]
                                p2=total_positions[token[1]]
                            
                                documents = intersect_prox(p1,p2,k)
                                if documents:
                                    results.append([doc,snip])

                                token.remove(token[0])
                                proximity.remove(proximity[0])
                            else:
                               
                                if i<len(terms)-1:
                                    term = terms[i+1]
                                elif i==len(terms)-1:
                                    term = terms[i]

                                token.append(term)

                            if (len(token) == 3):
                                token.remove(token[0])

                    elif  '/'  not in query and  "\"" not  in query:
                         print("Query Type: Free Text Query")
                         print(" free Query: found in document {},  snippet {}".format(doc,snip))
    return results







In [40]:
### AND OPEARTION ON QUERY
'''
res={}
min_word=''
min_count=100000
for keyword in keywords:
  c=0
  for doc in inverted_index[keyword].keys():
    c+=len(inverted_index[keyword][doc].keys())
  if(min_count>c):
    min_word=keyword
    min_count=c
for doc in inverted_index[min_word].keys():
  for snippet in inverted_index[min_word][doc].keys():
    same=1
    for keyword in keywords:
      if(doc in inverted_index[keyword].keys()) and (snippet in inverted_index[keyword][doc].keys()):
        continue
      else:
        same=0
        break
    if(same==1):
      if(doc in res.keys()):
        res[doc].append(snippet)
      else:
        res[doc]=[]
        res[doc].append(snippet)'''

"\nres={}\nmin_word=''\nmin_count=100000\nfor keyword in keywords:\n  c=0\n  for doc in inverted_index[keyword].keys():\n    c+=len(inverted_index[keyword][doc].keys())\n  if(min_count>c):\n    min_word=keyword\n    min_count=c\nfor doc in inverted_index[min_word].keys():\n  for snippet in inverted_index[min_word][doc].keys():\n    same=1\n    for keyword in keywords:\n      if(doc in inverted_index[keyword].keys()) and (snippet in inverted_index[keyword][doc].keys()):\n        continue\n      else:\n        same=0\n        break\n    if(same==1):\n      if(doc in res.keys()):\n        res[doc].append(snippet)\n      else:\n        res[doc]=[]\n        res[doc].append(snippet)"

In [41]:
### OR OPEARTION ON QUERY
def OR(keywords):
  res={}
  for keyword in keywords:
    for doc in inverted_index[keyword].keys():
      if(doc not in res):
        res[doc]=[]
      res[doc]=list(set(res[doc]+list(inverted_index[keyword][doc].keys())))
  return res

In [42]:
#TF-IDF vector
def generateVectors(keywords, res,ret_count):
    tf_idf_matrix = np.zeros((len(keywords), ret_count))
    for i, s in enumerate(keywords):
        idf = math.log(doc_count/df[s])+1
        j=0
        for doc in res.keys():
          for snippet in res[doc]:
            try:
              tf_idf_matrix[i][j] = idf * tf1[doc_to_id[doc]][snippet][s]
            except:
              tf_idf_matrix[i][j] =0
            j+=1
    return tf_idf_matrix

In [43]:
#Vector Query

def wcount(s):
    counts = dict()
    words = s
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts


def build_query_vector(keywords, res):
    count = wcount(keywords)
    vector = np.zeros((len(count),1))
    for i, word in enumerate(keywords):
          vector[i] = float(count[word])/len(count) * (1+math.log(doc_count/df[word]))
    return vector
    


In [44]:
def consine_similarity(v1, v2):
    return np.dot(v1,v2)/float(np.linalg.norm(v1)*np.linalg.norm(v2))
def compute_relevance(query, documents,tf_idf_matrix,query_vector,res):
    results=[]
    c=0
    for doc in res.keys():
      for snippet in res[doc]:
        similarity = consine_similarity(tf_idf_matrix[:,c].reshape(1, len(tf_idf_matrix)), query_vector)
        l=[doc_to_id[doc],snippet, float(similarity[0])]
        results.append(l)
        #print("document {}, snippet {}, similarity {}".format(doc_to_id[doc],snippet, float(similarity[0])))
        c+=1
    return results
        


 **USER** **SEARCH**


In [45]:
def query_func():
  query=""
  while query=="":
    query= input('Enter the query:')
    global start_time
    start_time=time.time()
  if '*' in query:
      keywords=wild_query(query)
  else:
      keywords=phrase_proximity_freetext_query(query,inverted_index)
  return keywords

# if in " ": phrase query
# if /k present: proximity query
# if "*" present: wildcard query
# if free text: (OR) of all the terms

In [48]:
results=[]
start_time=0
def main():
  keywords=[]
  keywords=query_func()
  print(keywords)
  try:
    res=OR(keywords)
    ret_count=0
    for doc in res.keys():
      ret_count+=len(res[doc])  
    tf_idf_matrix = generateVectors(keywords, res,ret_count)
    query_vector = build_query_vector(keywords, res)
    results=compute_relevance(words,res,tf_idf_matrix,query_vector,res)
    results=sorted(results,key=lambda l:l[2], reverse=True)
    c=1
    print("Response_time: ",time.time()-start_time)
    for res in results:
      if(c==11):
        break
      print("Document :",c)
      c+=1
      df=pd.read_csv(res[0])
      print("Similarity : ",res[2])
      for col in df.columns:
        print(col," : ",df[col][res[1]])
  except:

    print("Response_time: ",time.time()-start_time)
    for res in keywords:
      doc= doc_to_id[res[0]]
      print("Document :",doc)
      df = pd.read_csv(doc)
    
      for col in df.columns:
        print(col," : ",df[col][res[1]])



main()

Enter the query:"google"
[[4, 49], [25, 0], [31, 47], [32, 39], [49, 83], [59, 28], [59, 885], [59, 996], [59, 1043], [59, 1259], [69, 933], [69, 936], [76, 843], [76, 1446], [79, 632], [80, 202], [80, 992], [80, 1090], [96, 13], [145, 103], [146, 13], [146, 14], [166, 17], [170, 327], [170, 328], [170, 329], [170, 330], [170, 361], [170, 362], [170, 442], [170, 443], [172, 84], [195, 50], [195, 51], [212, 45], [250, 39], [258, 51], [258, 75], [258, 99], [276, 13], [284, 193], [284, 205], [287, 12], [287, 172], [303, 5], [303, 6], [303, 29], [305, 28], [323, 210], [333, 263], [333, 290], [333, 291], [333, 303], [338, 143], [338, 213], [351, 186], [366, 248], [374, 149], [384, 140], [388, 115], [394, 12], [394, 18], [415, 704]]
Response_time:  0.0013194084167480469
Document : /content/drive/My Drive/AIR/TelevisionNews/CNN.201004.csv
URL  :  https://archive.org/details/CNN_20100429_170000_CNN_Newsroom#start/1100/end/1135
MatchDateTime  :  4/29/2010 17:18:35
Station  :  CNN
Show  :  CNN N

In [ ]:
keywords

[]

Creating Index for ES

In [ ]:
#es.indices.delete(index="news")
es.indices.create(index="news")

In [ ]:
#uploading files to ELASTIC SEARCH
folder="/content/drive/My Drive/AIR/TelevisionNews/"
count=1
for i in glob.glob(folder+"/*csv"):
    path=i
    try:
        df=pd.read_csv(path)
        for row in df.index:
            data={}
            for col in df.columns:
                if(len(df[col][row])!=0):
                    data[col]=df[col][row]
                else:
                    data[col]="empty"
            data["doc"]=path
            data["snippet"]=row
            try:
                es.index(index="news", doc_type="word", id=count, body=data)
            except:
                print(path," ",row)
            count+=1
    except:
        continue

In [ ]:
#ELASTIC SEARCH
body = {
    "from":0,
    "size":10000,
    "query": {
        "match": {
            "Snippet":query
        }
    }
}

ground_truth = es.search(index="news", body=body)

In [ ]:

#EVALUATION METRIC
tp=0
for i in ground_truth['hits']['hits']:
  for j in results: 
    if(i['_source']['doc']==j[0] and i['_source']['snippet']==j[1]):
      tp+=1
      break
fp=len(results)-tp
fn=len(ground_truth['hits']['hits'])-tp
precision=tp/(tp+fp)
recall=tp/(tp+fn)

In [ ]:
#MAP
avg_precision=[]
num=0
den=0
for i in results:
  found=False
  for i in ground_truth['hits']['hits']:
    if(i['_source']['doc']==j[0] and i['_source']['snippet']==j[1]):
      num+=1
      den+=1
      avg_precision.append(num/den)
      found=True
      break
    if(found==False):
      den+=1
      avg_precision.append(num/den)